# Reinforcement Learning Lab

In this lab we study exploration in the very abstract $n$-armed bandit ask. In this there are $n$ actions to take. Each returns a reward $R$, with some probability $p$. The reward value is either a 1 or a 0. This means the expected value of each arm is simply its probability of reward. Nice and simple right?

Our agents are really learning, at last. Reinforcement learning, to be precise. 

The reward value $Q$ update rule for all agents (below) and arms is the same:

$$ Q \leftarrow Q + \alpha * (R - Q) $$ [1]

Where the learning rate _lr_reward_ is denoted as $\alpha$, so that the equation above looks nice. The learning rate often gets called $\alpha$ in the reinforcement learning world. If you are not familiar with the idea of a learning rate, it is what it sounds like. A parameter that controls how much each value update matters. This is, over time, the rate at which learning happens.

Remember too that $Q$ is trying to approximate the average reward value of each arm.

This kind of difference $(R - Q)$ in Eq [1] is typical of RL. If you're not sure what it means, consider in your head, what would happen to the value update if $Q$ was bigger than the reward $R$ (and overestimate), or if it was smaller. Once you have noodled that a bit, as needed, consider how making $\alpha$ bigger or smaller might make $Q$ learning faster, or slower, or more or less volatile. (Learning speed and volatility _often_ go together; an annoying matched set.)

_Note_: We are not going to use $\alpha$ here. Just giving you some intuition.

We will first be considering three basic agents:
- A random agent that picks its actions with a uniform probability for all 4 arms.  Each action is selected independently.
- A sequential agent that samples the arms evenly by cycling through its options
- An $\epsilon$-greedy, which picks the "best" arm $1-\epsilon$ of the time, and picks uniformly randomly from the other arms $\epsilon$ of the time.

The first two agents do not have any sense of exploitation, but we can create an exploitative agent by putting a bound on their exploration.  The *bounded* random and *bounded* sequential agents explore for a fixed number of trials (depending on the `bound` we set) according to their respective strategy, and once the bound is exceeded, they will always pick the arm with the highest Q value.

The lab has two sections. 

- _First_, we will explore 4-armed bandits, and get to know our three basic agents.

- _Second_, is where things get more interesting. We'll put $\epsilon$=greedy, and the two bounded agents, in a little competition. 

Our metric is _total reward_. Maximizing that is the goal of all RL, afterall.
.


## Install and import needed modules


In [ ]:
# Install explorationlib?
!pip install --upgrade git+https://github.com/clappm/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git

In [ ]:
# import basic modules
import shutil
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# import explorationlib
import explorationlib

# import the type of environment we will be using
from explorationlib.local_gym import BanditUniform4

# import the components to build our agents
from explorationlib.agent import BanditActorCritic
from explorationlib.agent import Critic
from explorationlib.agent import EpsilonActor
from explorationlib.agent import RandomActor
from explorationlib.agent import SequentialActor
from explorationlib.agent import BoundedRandomActor
from explorationlib.agent import BoundedSequentialActor

# import the experimental framework
from explorationlib.run import experiment

# import some scoring functions
from explorationlib.score import total_reward
from explorationlib.score import action_entropy

# import some utility functions
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

# import some plotting functions
from explorationlib.plot import plot_bandit
from explorationlib.plot import plot_bandit_actions
from explorationlib.plot import plot_bandit_critic
from explorationlib.plot import plot_bandit_hist

In [ ]:
# Pretty plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

# Dev
%load_ext autoreload
%autoreload 2

## Section 1.1 - The Environment

Let's look some at the code defining this class of environment.  As you might guess from the import statement, it is found in local_gym.py.

*show how to nagivate there in colab*

For convience, the class BanditUniform4 (and its parent class, BanditEnv) is copy-pasted below:

In [ ]:
# ---------------------------------------------------------------------------
# Basic Bandits
# - A base class,
# - then several working examples
# ---------------------------------------------------------------------------
class BanditEnv(gym.Env):
    """
    n-armed bandit environment  

    Params
    ------
    p_dist : list
        A list of probabilities of the likelihood that a particular bandit 
        will pay out
    r_dist : list or list or lists
        A list of either rewards (if number) or means and standard deviations
        (if list) of the payout that bandit has
    """
    def __init__(self, p_dist, r_dist):
        if len(p_dist) != len(r_dist):
            raise ValueError(
                "Probability and Reward distribution must be the same length")

        if min(p_dist) < 0 or max(p_dist) > 1:
            raise ValueError("All probabilities must be between 0 and 1")

        for reward in r_dist:
            if isinstance(reward, list) and reward[1] <= 0:
                raise ValueError(
                    "Standard deviation in rewards must all be greater than 0")

        self.p_dist = p_dist
        self.r_dist = r_dist

        self.n_bandits = len(p_dist)
        self.action_space = spaces.Discrete(self.n_bandits)
        self.observation_space = spaces.Discrete(1)
        self.seed()

    def step(self, action):
        assert self.action_space.contains(action)
        self.state = 0
        self.reward = 0
        self.done = False

        if self.np_random.uniform() < self.p_dist[action]:
            if not isinstance(self.r_dist[action], list):
                self.reward = self.r_dist[action]
            else:
                self.reward = self.np_random.normal(self.r_dist[action][0],
                                                    self.r_dist[action][1])

        return self.state, self.reward, self.done, {}

    def last(self):
        return self.state, self.reward, self.done, {}

    def reset(self):
        self.state = 0
        self.reward = 0
        self.done = False

    def render(self, mode='human', close=False):
        pass


class BanditUniform4(BanditEnv):
    """A 4 armed bandit."""
    def __init__(self, p_min=0.1, p_max=0.3, p_best=0.6, best=2):
        self.best = [best]
        self.num_arms = 4

        # ---
        self.p_min = p_min
        self.p_max = p_max
        self.p_best = p_best

        # Generate intial p_dist
        # (gets overwritten is seed())
        p_dist = np.random.uniform(self.p_min, self.p_max,
                                   size=self.num_arms).tolist()
        p_dist[self.best[0]] = self.p_best

        # reward
        r_dist = [1] * self.num_arms

        # !
        BanditEnv.__init__(self, p_dist=p_dist, r_dist=r_dist)

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)

        # Reset p(R) dist with the seed
        self.p_dist = self.np_random.uniform(self.p_min,
                                             self.p_max,
                                             size=self.num_arms).tolist()
        self.p_dist[self.best[0]] = self.p_best

        return [seed]

Let's make a four armed bandit and then plot the expected value of each arm.

_Note_: The random seed is fixed. If you change the seed and run the cell below, some of the reward probabilities will change. The probability of the best arm, the optimal value arm is fixed however. It is set to 0.35, and located at arm 2. Try it! Rerun the cell below with different seeds, a few times, to get a sense of how the non-optimal arms can vary.

In [ ]:
# Shared env params
seed = 503

# Create env
env = BanditUniform4(p_min=0.1, p_max=0.3, p_best=0.35)
env.seed(seed)

# Plot env
plot_bandit(env, alpha=0.6)

## Section 1.2 - Our three agents, unbounded.

### Our three agents, _unbounded_

A word about the code. Our agents this week work in whaat gets called an ActorCritic desgin. This breaks reinforcement learning algorithms into two parts: The Actor does action selection. The Critic estimates the value of each action.

Now in normal reinforcement learning, aka not pure exploration, the _Actor_ uses the $Q$ value estimates from the _Critic_ to, in part, make its decisions. Be it explore or exploit. This is indeed the case for the $\epsilon$-greedy agent, _EpsilonActor_, works. 

...But... 

The other two agents--_SequentialActor_ and _RandomActor_--don't explore with value. The are both _max entropy_ action systems, who don't care about reward value or learning _at all_. We have kept the _ActorCritic_ style because it was easy to implement in _explorationlib_. Don't be misled.

Below are the class definitions for some of the important classes.

In [ ]:
class BanditActorCritic(BanditAgent):
    """Actor-Critic agent"""
    def __init__(self, actor, critic, lr_reward=0.1):
        super().__init__()

        self.actor = actor
        self.critic = critic
        self.lr_reward = float(lr_reward)

    def __call__(self, state):
        return self.forward(state)

    def update(self, state, action, R, next_state, info):
        self.critic_R = R_update(action, R, self.critic, self.lr_reward)

    def forward(self, state):
        action = self.actor(list(self.critic.model.values()))
        return action

    def reset(self):
        self.actor.reset()
        self.critic.reset()

In [ ]:
class RandomActor(BanditAgent):
    def __init__(self, num_actions):
        super().__init__()
        self.num_actions = num_actions
        self.actions = list(range(self.num_actions))

    def __call__(self, values):
        return self.forward(values)

    def forward(self, values):
        """Values are a dummy var. Pick at random"""
        action = self.np_random.choice(self.actions)
        return action

    def reset(self):
        pass

In [ ]:
class SequentialActor(BanditAgent):
    """Choose actions in sequence; ignore value"""
    def __init__(self, num_actions, initial_action=0):
        super().__init__()

        self.num_actions = int(num_actions)
        self.initial_action = int(initial_action)
        self.action_count = self.initial_action

    def __call__(self, values):
        return self.forward(values)

    def forward(self, values):
        self.action_count += 1
        action = self.action_count % self.num_actions

        return action

    def reset(self):
        self.action_count = self.initial_action

In [ ]:
class EpsilonActor(BanditAgent):
    def __init__(self, num_actions, epsilon=0.1, decay_tau=0.001):
        super().__init__()

        self.epsilon = epsilon
        self.decay_tau = decay_tau
        self.num_actions = num_actions

    def __call__(self, values):
        return self.forward(values)

    def decay_epsilon(self):
        self.epsilon -= (self.decay_tau * self.epsilon)

    def forward(self, values):
        # If values are zero, be random.
        if np.isclose(np.sum(values), 0):
            action = self.np_random.randint(0, self.num_actions, size=1)[0]
            return action

        # Otherwise, do Ep greedy
        if self.np_random.rand() < self.epsilon:
            action = self.np_random.randint(0, self.num_actions, size=1)[0]
        else:
            action = np.argmax(values)

        return action

    def reset(self):
        pass

The most important code defining the critic, which handles the Q value update rule:

In [ ]:
def R_update(state, R, critic, lr):
    """TD-ish update"""
    update = lr * (R - critic(state))
    critic.update(state, update)

    return critic

class Critic(BanditAgent):
    """Template for a Critic agent"""
    def __init__(self, num_inputs, default_value=0.0):
        super().__init__()

        self.num_inputs = num_inputs
        self.default_value = default_value

        self.model = OrderedDict()
        for n in range(self.num_inputs):
            self.model[n] = self.default_value

    def __call__(self, state):
        return self.forward(state)

    def forward(self, state):
        return self.model[state]

    def update(self, state, update):
        self.model[state] += update

    def replace(self, state, update):
        self.model[state] = update

    def state_dict(self):
        return self.model

    def load_state_dict(self, state_dict):
        self.model = state_dict

    def reset(self):
        self.model = OrderedDict()
        for n in range(self.num_inputs):
            self.model[n] = self.default_value

Now we can initialize out three basic agents:

In [ ]:
ran = BanditActorCritic(
    RandomActor(num_actions=env.num_arms),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)
seq = BanditActorCritic(
    SequentialActor(num_actions=env.num_arms),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)
epy = BanditActorCritic(
    EpsilonActor(num_actions=env.num_arms, epsilon=0.1),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)

# organize them
agents = [ran, seq, epy]
names = ["random", "sequential", "ep-greedy"]
colors = ["blue", "green", "purple"]

## Section 1.3 - Running Simulations

Let's run out our three agents on the environment from before and plot some things about them.

In [ ]:
num_steps = 12  # Three rounds per arm, about that anyway
num_experiments = 1

# !
results = []
for name, agent in zip(names, agents):
    log = experiment(
        f"{name}",
        agent,
        env,
        num_steps=num_steps,
        num_experiments=num_experiments,
        dump=False,
        split_state=False,
    )
    results.append(log)

### Plot action choices 
with time (aka steps).

In [ ]:
num_experiment = 0
for name, res, color in zip(names, results, colors):
    plot_bandit_actions(
        select_exp(res, num_experiment), 
        num_arms=4,
        s=4,
        title=name, 
        color=color,
        figsize=(2, 1.5)
        )

### Plot histograms of action probability (aka arm choice). 

_Note_: The flatter these plots are, the closer they are to _maximum entropy_ exploration behavior. I added a measure of the actual entropy to the title of each plot, to make it easier to compare.

In [ ]:
num_experiment = 0
ax = None
for name, res, color in zip(names, results, colors):
    ent = np.round(np.mean(action_entropy(res)), 2)
    plot_bandit_hist(
        select_exp(res, num_experiment), 
        bins=list(range(0, 5)),
        title=f"{name} (ent {ent})", 
        alpha=0.4,
        color=color,
        figsize=(3, 3),
        ax=ax
        )

## Section 1.4 - Meet our dilemma
Should I explore? Should I exploit it!? 

....I'll flip a weighted coin, 

...whose weight has a name. It's $\epsilon$! 

The smaller $\epsilon$ is, the less likely the coin flip comes up "EXPLORE'' and the more likely it comes up on the "EXPLOIT" side. If one chooses the exploit side, one is being greedy, right? The bigger $\epsilon$ the more likely the coin will say "EXPLORE ''. Etc.

Let's play with $\epsilon$-greedy.

In [ ]:
num_steps = 4 * 100
epsilons = [0.05, 0.5, 0.95]

names = [str(epsilon) for epsilon in epsilons]
colors = ["mediumpurple", "mediumorchid", "mediumvioletred"]

# !
results = []
for i, (name, epsilon) in enumerate(zip(names, epsilons)):
    agent = BanditActorCritic(
        EpsilonActor(num_actions=env.num_arms, epsilon=epsilon),
        Critic(num_inputs=env.num_arms, default_value=0.0)
    )
    log = experiment(
        f"ep_{name}",
        agent,
        env,
        num_steps=num_steps,
        num_experiments=100,
        dump=False,
        split_state=False,
    )
    results.append(log)

Example behave. Change _num experiment_ to see more examples (0, 99). 

_Note_: in every experiment we run in this lab, the optimal value arm is _always_ arm 2.

In [ ]:
num_experiment = 40
for name, res, color in zip(names, results, colors):
    plot_bandit_actions(
        select_exp(res, num_experiment), 
        max_steps=200,
        s=4,
        title=name, 
        color=color,
        figsize=(3, 1)
        )

Total reward 

In [ ]:
# Score
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)   

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.8)
plt.ylabel("Total reward")
plt.xlabel("Epsilon")
plt.tight_layout()
sns.despine()

In [ ]:
fig = plt.figure(figsize=(6, 3))
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.8, bins=np.linspace(1, 200, 50))
    plt.legend()
    plt.xlabel("Total reward")
    plt.ylabel("Count")
    plt.tight_layout()
    sns.despine()

Action entropy

In [ ]:
# Score
scores = []
for name, res, color in zip(names, results, colors):
    r = action_entropy(res)
    scores.append(r)   

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Entropy")
plt.xlabel("Epsilon")
plt.tight_layout()
sns.despine()

In [ ]:
fig = plt.figure(figsize=(6, 3))
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.8, bins=np.linspace(0, 1.6, 40))
    plt.legend()
    plt.xlabel("Entropy")
    plt.ylabel("Count")
    plt.tight_layout()
    sns.despine()

## Section 2.1 - Bounds and bandits

In this section we'll study two pure explorers with bounds. Our friend $\epsilon$-greedy returns, unchanged.

Let's tune the bound parameter of the bounded agents, assuming 400 steps is an ok number.

In [ ]:
num_steps = 4 * 100
num_experiments = 250

In [ ]:
# Defining environment
# DO NOT CHANGE:

# Shared env params
num_experiments = 100
seed = 593  

# Create env
env = BanditUniform4(p_min=0.1, p_max=0.3, p_best=0.35)
env.seed(seed)

# Plot env
plot_bandit(env, alpha=0.6)

## Section 2.2 - Tuning the bound.

What is the best _bound_ (for this task)?

The code below is a first try at tuning _BoundedRandomActor_ and its only parameter _bound_. Run it. Then use the code to try and narrrow down a best choice for this environment. Put each attempt you make in a new cell, as a way to show your work. (Do not change _num steps_ or _num experiments_, above).

Note: we are only going to tune the bound for BoundedRandomActor, and we'll recycle this for BoundedSequentialActor.

In [ ]:
# CHANGE ME
start = 4 # min 4
stop = num_steps # max num_steps
num_search = 10 # ?

# -
# LEAVE ME
bounds = np.linspace(start, stop, num_search).astype(int)
names = [str(np.round(bound, 2)) for bound in bounds]

# !
results = []
for i, (name, bound) in enumerate(zip(names, bounds)):
    agent = BanditActorCritic(
        BoundedRandomActor(num_actions=env.num_arms, bound=bound),
        Critic(num_inputs=env.num_arms, default_value=0.0)
    )
    log = experiment(
        f"b_{name}",
        agent,
        env,
        num_steps=num_steps,
        num_experiments=num_experiments,
        dump=False,
        split_state=False,
    )
    results.append(log)

# Score
scores = []
for name, res in zip(names, results):
    r = total_reward(res)
    scores.append(r)   

# Tabulate
m, sd = [], []
for (name, s) in zip(names, scores):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot
fig = plt.figure(figsize=(8, 3))
plt.bar(names, m, yerr=sd, color="black", alpha=0.8)
plt.ylabel("Total reward")
plt.xlabel("Bound")
plt.tight_layout()
sns.despine()

## Section 2.3 - Reporting Results

The tuned parameters are entered here:

In [ ]:
bound = 75
epsilon = 0.1

In [ ]:
# DO NOT CHANGE:
# Shared env params
seed = 195

# Create env
env = BanditUniform4(p_min=0.1, p_max=0.3, p_best=0.35)
env.seed(seed)

# Plot env
plot_bandit(env, alpha=0.6)

In [ ]:
bounded_ran = BanditActorCritic(
    BoundedRandomActor(num_actions=env.num_arms, bound=bound),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)
bounded_seq = BanditActorCritic(
    BoundedSequentialActor(num_actions=env.num_arms, bound=bound),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)
epy = BanditActorCritic(
    EpsilonActor(num_actions=env.num_arms, epsilon=epsilon),
    Critic(num_inputs=env.num_arms, default_value=0.0)
)

# organize them
agents = [bounded_ran, bounded_seq, epy]
names = ["b-ran", "b-seq", "ep-greedy"]
colors = ["blue", "green", "purple"]

In [ ]:
num_steps = 400  # Three rounds per arm, about that anyway
num_experiments = 250

# !
results = []
for name, agent in zip(names, agents):
    log = experiment(
        f"{name}",
        agent,
        env,
        num_steps=num_steps,
        num_experiments=num_experiments,
        dump=False,
        split_state=False,
    )
    results.append(log)

In [ ]:
# Score
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)   

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(5, 5))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.8)
plt.ylabel("Total reward")
plt.xlabel("Agent")
plt.tight_layout()
sns.despine()